In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt
from scipy.linalg import fractional_matrix_power

Let's  try to reproduce the idea of [this paper](http://faculty.olin.edu/dshuman/Papers/Conference/Thanou_et_al_GlobalSIP_2013.pdf).
In particular we are going to define a dictionary composed of $S$ subdictionaries, each of them parametrized as a polynomial of the laplacian: 

$$
D_s = \sum_{k = 0}^K \alpha_{sk} \mathcal{L}^k, \ s = 1,...,S
$$

this dictionary has the structure of a convolutional filter bank, and by looking at the spectrum of each subdictionary we find out that each atom of each subdictionary has a localization quality over a node $v \in V$.

Assuming a graph $G(V,E)$, and a set of $N$ observed signals $Y \in \mathbb{R}^{V \times N}$, we have the following optimization problem:

$$
\begin{cases}
\underset{X,D}{\mathrm{min}} & ||Y -DX||_F^2 + \lambda ||X||_{:,1}\\
& D_s = \sum_{k = 0}^K \alpha_{sk} \mathcal{L}^k, & s = 1,...,S \\
& 0 \preceq D_s \preceq k\mathbb{I}, & s = 1,...,S \\
& (c - \epsilon) \mathbb{I} \preceq \sum_{s=1}^S D_s \preceq (c + \epsilon)\mathbb{I}

\end{cases}
$$

where the constraints are meant to grant that the spectrum of each filter and of the over all dictionary meaningfully covers the representations of the signals. 

The problem can be solved alternating the optimization of $D$, i.e. of its parametrization, and of the sparse coding $X$ of each 0-cochain.

### Building the graph

In [2]:
N = 80
cutoff = 0.5
theta = 0.9

points = np.random.rand(N, 2)

In [3]:
def GaussianKernel(P1, P2, theta):

    return np.exp(- np.linalg.norm(P1 - P2)**2/(2*theta**2))

In [4]:
A = np.zeros((N,N))
W = np.zeros((N,N))

for i in range(N):
    for j in range(i, N):
        
        A[i,j] = np.linalg.norm(points[i,:] - points[j,:]) <= cutoff
        A[j,i] = np.linalg.norm(points[i,:] - points[j,:]) <= cutoff

        W[i,j] = GaussianKernel(points[i,:], points[j,:], theta)
        W[j,i] = GaussianKernel(points[i,:], points[j,:], theta)

W = W * A

In [5]:
D = np.diag(np.sum(A, axis = 1))

In [6]:
L = D - W 
L = fractional_matrix_power(D, -0.5) @ L @ fractional_matrix_power(D, -0.5)

### Define a band-localized overcomplete dictionary

In [16]:
band1 = np.array(list(range(0,20)))
band2 = np.concatenate([np.array(list(range(20,30))), np.array(list(range(70,80)))])
band3 = np.array(list(range(30,50)))
band4 = np.array(list(range(50,70)))

bands = [band1, band2, band3, band4]

In [32]:
Lambda, U = np.linalg.eig(L)

In [42]:
J = 320
DD = np.zeros((N, J))

In [43]:
for j in range(J): 

    B = np.random.choice(4)

    h = np.copy(Lambda)
    h[~bands[B]] = 0
    h *= np.random.rand(h.shape[0],1)[:,0]

    n = np.random.choice(N)

    DD[:,j] = (U @ np.diag(h) @ U.T)[:,n]

### Define the signals 

In [53]:
M = 1000 
Y_train = np.zeros((N,M))

for m in range(M):
    T = np.random.choice(J, 4)
    Y_train[:,m] = DD[:,T] @ np.random.rand(4)

test_size = 2000
Y_test = np.zeros((N,test_size))

for t in range(test_size):
    T = np.random.choice(J, 4)
    Y_test[:,t] = DD[:,T] @ np.random.rand(4)

### Dictionary learning algorithm 

In [ ]:
c = 1
eps = 0.01

T0 = 4
S = 4
K = 20